<a href="https://colab.research.google.com/github/mmmhdp/nn_labs/blob/main/lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [127]:
import torch as tr
import torchvision as tv
import pandas as pd
import numpy as np
import scipy as scp
import datetime as dt
from torchvision.transforms import v2

from torchsummary import summary
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import pyplot as plt
import seaborn as sns

# model stuff


In [128]:
mnist_to_lenet_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(32),
])

mnist_trainset = tv.datasets.MNIST(root='./data',
                                   train=True,
                                   download=True,
                                   transform=mnist_to_lenet_transforms)
mnist_test = tv.datasets.MNIST('./data',
                               train=False,
                               download=True,
                               transform=mnist_to_lenet_transforms)

batch_size = 128
mnist_train_dataloader = DataLoader(mnist_trainset, batch_size=batch_size)
mnist_test_dataloader = DataLoader(mnist_test, batch_size=batch_size)

In [129]:
mnist_trainset, mnist_test

(Dataset MNIST
     Number of datapoints: 60000
     Root location: ./data
     Split: Train
     StandardTransform
 Transform: Compose(
                  ToImage()
                  Resize(size=[32], interpolation=InterpolationMode.BILINEAR, antialias=warn)
            ),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: ./data
     Split: Test
     StandardTransform
 Transform: Compose(
                  ToImage()
                  Resize(size=[32], interpolation=InterpolationMode.BILINEAR, antialias=warn)
            ))

In [157]:
nclasses = 10
finelenet = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=0),
            nn.Tanh(),
            nn.AvgPool2d(2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5),
            nn.Flatten(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 10),

            nn.ReLU(),
            nn.Linear(10, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, nclasses),
            )

finelenet

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): Tanh()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Tanh()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=120, out_features=84, bias=True)
  (9): Tanh()
  (10): Linear(in_features=84, out_features=10, bias=True)
  (11): ReLU()
  (12): Linear(in_features=10, out_features=256, bias=True)
  (13): ReLU()
  (14): Linear(in_features=256, out_features=512, bias=True)
  (15): ReLU()
  (16): Linear(in_features=512, out_features=256, bias=True)
  (17): ReLU()
  (18): Linear(in_features=256, out_features=10, bias=True)
)

In [158]:
epoch_amount = 100

network = finelenet
opt = tr.optim.RMSprop(network.parameters(), lr=1e-5)
sch = tr.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=10)
device = tr.device("cuda") if tr.cuda.is_available() else tr.device("cpu")
loss_f = nn.CrossEntropyLoss()

def evaluate_accuracy(data_iter, net):
    acc_sum, n = tr.Tensor([0]), 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n


for epoch in range(epoch_amount):
    network.train()

    for features_batch, target_batch in mnist_train_dataloader:
        opt.zero_grad()

        features_batch = features_batch.to(device).float()
        target_batch = target_batch.to(device)

        pred_target = network(features_batch)

        loss = loss_f(pred_target, target_batch)
        loss.backward()
        opt.step()

        sch.step()

    network.eval()

KeyboardInterrupt: 

In [159]:
x,y = next(iter(mnist_test_dataloader))
pr_x = network(x.float())

In [160]:
accuracy_score(y_true=y, y_pred=tr.tensor([tr.argmax(el).item() for el in pr_x]))

0.84375